In [71]:
from importlib import reload
import nbimporter
from collections import Counter
import functions_readbedfiles, functions_plot
from functions_plot import create_data_traintest, create_data4T_SNE
from Functions_ReadBedFiles import readJsonFile, readFiles2Vector, writeJsonFile, convertMat2document, readJsonFile
import glob
import pandas as pd
import numpy as np
import time
import copy
import gc
from gensim.models import Word2Vec
from sklearn.decomposition import PCA

Importing Jupyter notebook from functions_readbedfiles.ipynb
Importing Jupyter notebook from functions_plot.ipynb


In [59]:
import numpy as np
from gensim.models import Word2Vec
import json

In [60]:
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import matplotlib.font_manager as font_manager

In [72]:
from representations_regionset2vec import document_embedding_avg

Importing Jupyter notebook from representations_regionset2vec.ipynb


In [62]:
def initialization(clas_type, label, path_train, path_test, path_universe, path_mat, meta_data, sample_of_interest, path_w2v_model):
    path_train = path_train.format(clas_type)
    path_test = path_test.format(clas_type)
    path_universe = path_universe.format(clas_type)
    path_mat = path_mat
    meta_data = pd.read_csv(meta_data.format(clas_type))
    meta_data = meta_data.loc[meta_data[label].isin(sample_of_interest)][['Experiment_ID', label]]
    model = Word2Vec.load(path_w2v_model.format(clas_type))
    
    return path_train, path_test, path_universe, path_mat, meta_data, model

# Read data

In [65]:
i = 1

In [66]:
clas_types = ['antibody', 'cell', 'tissue']
labels = ['antibody', 'cell line', 'tissue']
sample_of_interest = [['h3k27ac', 'h3k4me3', 'h3k27me3', 'h3k4me1', 'h3k36me3', 'h3k9me3', 'h3k4me2'],
                     ['k562', 'mcf7', 'hek293', 'a549', 'hepg2', 'hct116','lovo', 'gm12878', 'lncap','hela'],
                     ['liver', 'peripheral blood', 'primary prostate cancer', 'blood', 'breast','bone marrow', 'kidney']
                     ]
PCA_flg = False
label = labels[i]
clas_type = clas_types[i]

In [67]:
results = pd.DataFrame(columns = ['dataset','representation',  'pca', 'f1'])

In [73]:
path_train = '/project/shefflab/www/papers/region_embedding/{}dataset/train/*'
path_test = '/project/shefflab/www/papers/region_embedding/{}dataset/test/*'
path_universe = './representations/{}/'
path_mat = '/project/shefflab/data/ChIP-Atlas/term_doc_mat/'
meta_data = './meta_data/meta_data_{}.csv'
path_w2v_model = './region_embedding_models/word2vec_{}.model'

In [69]:
path_train, path_test, path_universe, path_mat, meta_data, model = initialization(clas_type, label, path_train, path_test, path_universe, path_mat, meta_data, sample_of_interest[i], path_w2v_model)


In [70]:
%%time

for path_univ in sorted(glob.glob(path_universe + "*")):
    gc.collect()
    pca = PCA(n_components= 100)
    print(path_univ)
    train_files , segmentation_df_train = readFiles2Vector(path_train, path_univ, numberofCores = 4, numOfFiles= 100, PATH = path_train)
    print(len(train_files))
    
#     segmentation_df_train = None
    
    test_files, segmentation_df_test = readFiles2Vector(path_test, path_univ, numberofCores = 4, numOfFiles= 100, PATH = path_test)
    print(len(test_files))
    
#     segmentation_df_test = None

    X_train, y_train = create_data4T_SNE(train_files, (meta_data.loc[(meta_data[label] !='none') & (meta_data[label] !='nan') & (meta_data[label].notna())]), label, 0, 'Experiment_ID')
    print(len(X_train), len(y_train))
    X_test, y_test = create_data4T_SNE(test_files, (meta_data.loc[(meta_data[label] !='none') & (meta_data[label] !='nan') & (meta_data[label].notna())]), label, 0, 'Experiment_ID')
    print(len(X_test), len(y_test))
    print('Done')
    
    if(PCA_flg == True):
        X_train = pca.fit_transform(X_train)
        X_test = pca.transform(X_test)
#     print(len(X_train[0]), len(X_test[0]))
    
#     clf = RandomForestClassifier(max_depth= 10, random_state=10)
    clf = svm.SVC(kernel = 'linear')
    start_time = time.time()
    clf.fit(X_train, y_train)

    f1 = f1_score((y_test), clf.predict(X_test), average = 'micro')
    print(f1_score((y_test), clf.predict(X_test), average = 'micro'))
    results = pd.concat([results, pd.DataFrame([[clas_type, path_univ.split('/')[-1], PCA_flg, f1]], columns= ['dataset','representation',  'pca', 'f1'])], ignore_index=True)
    


document_json_train = '/project/shefflab/data/ChIP-Atlas/term_doc_mat/document_{}_universe-txt_train.json'.format(clas_type)
with open(document_json_train, 'r') as j:
    document_Embedding_train =  json.loads(j.read())
    
document_json_test = '/project/shefflab/data/ChIP-Atlas/term_doc_mat/document_{}_universe-txt_test.json'.format(clas_type)
with open(document_json_test, 'r') as j:
    document_Embedding_test =  json.loads(j.read())
    

document_Embedding_avg_train = document_embedding_avg(document_Embedding_train, model)
document_Embedding_avg_test = document_embedding_avg(document_Embedding_test, model)
    
X_train, y_train = create_data4T_SNE(document_Embedding_avg_train, (meta_data.loc[(meta_data[label] !='none') & (meta_data[label] !='nan') & (meta_data[label].notna())]), label, 0, 'Experiment_ID')
X_test, y_test = create_data4T_SNE(document_Embedding_avg_test, (meta_data.loc[(meta_data[label] !='none') & (meta_data[label] !='nan') & (meta_data[label].notna())]), label, 0, 'Experiment_ID')

print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

# clf = RandomForestClassifier(max_depth= 10, random_state=10)
clf = svm.SVC(kernel = 'linear')
start_time = time.time()
clf.fit(X_train, y_train)

f1 = f1_score((y_test), clf.predict(X_test), average = 'micro')
print(f1_score((y_test), clf.predict(X_test), average = 'micro'))
results = pd.concat([results, pd.DataFrame([[clas_type, 'Region-set2vec', PCA_flg, f1]], columns= ['dataset', 'representation',  'pca', 'f1'])], ignore_index=True)

results.to_csv('./Results/F1_class{}_pca{}.csv'.format(clas_type, PCA_flg), index = False)


5527 5527
1368 1368
0.9605263157894737
CPU times: user 8min 1s, sys: 3.41 s, total: 8min 5s
Wall time: 8min 49s
